# This script identifies and deletes point features that have an identical attribute (settlement name) within a predefined distance of each other.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import os
from math import radians, sin, cos, acos
import arcpy


# Read data and define variables

In [2]:
#df = gpd.read_file(r'C:\Users\Kelly\Documents\GIS_COLUMBIA\SSD\SSD_Settlements\SharedWithPartners\SSD_StlPnt_V01_alpha\SSD_StlPnt_V01_alpha.shp')
dataDir = r'C:\Users\Kelly\Documents\ArcGIS\Projects\ssdSttl\ssdSttl.gdb'
df = gpd.read_file(dataDir, driver = 'FileGDB', layer = 'grid_reach_layer1_2_3_all')

data_path = r'C:\Users\Kelly\Documents\GIS_COLUMBIA\SSD\SSD_Settlements'

pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', None)

# The column names used in the script are:
longitude cloumn = x; latitude column = y


The column that contains possible duplicate attributes is "name".

Insert the field names, which are in your input shapefile, as dataframe column names below on the right side under #define your variables.

The variables to the left are those used by the script. Do not change those.

Define the threshold distance (meter). The script will search for duplicate names within this distance.

In [3]:
#define your variables on the right
df['x'] = df['long']    #longitude values of the point locations
df['y']  = df['lat']    #latitude vaues of the point locations
df['name'] = df['name'] #place names of the locations

#Define the threshold distance (meter). The script will search for duplicate names within this distance.
threshold = 500

#Explore your data. Print row- and column counts.
df.shape

(30677, 22)

# Define function that will calculate distances between points in meter

In [4]:
def haversine(lon1, lat1, lon2, lat2):
    # convert degrees to radians
    lon1 = np.deg2rad(lon1)
    lat1 = np.deg2rad(lat1)
    lon2 = np.deg2rad(lon2)
    lat2 = np.deg2rad(lat2)

    # formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    r_e = 6371
    return c * r_e*1000

# Run function and calculate distances between points with identical name

In [5]:
#add index column as reference for future query and join
df['index_col'] = df.index

#Code merge
# merge dataframe on settlement names
m = df.reset_index().merge(df.reset_index(), on='name')

# Calculate Distance
m['distance'] = haversine(m.x_x, m.y_x, m.x_y, m.y_y)


# Calculate midpoint coordinates and create unique identifier of the midpoints

In [6]:
m['Xmid']= (m['x_y']+m['x_x'])/2
m['Ymid']= (m['y_y']+m['y_x'])/2

#concatenate midpoint x,y values to create unique identifiers
m['MID'] = m['Xmid'].astype(str) + m['Ymid'].astype(str)


# Select distances less or equal threshold distance

In [7]:
m.drop(m[m.distance == 0].index, inplace=True)

#specify threshold distance. Beyond this distance all duplicate names will be deleted.
m.drop(m[m.distance > threshold].index, inplace=True) 

#print(m.head())


# Save duplicate locations that sould be deleted as shapefile, if you would like to explore those later on

In [8]:
df1 = m[['index_x', 'name', 'lat_x','long_x','distance','MID', 'index_col_x']].copy()
gdf = gpd.GeoDataFrame(df1, geometry=gpd.points_from_xy(df1.long_x, df1.lat_x))
gdf.to_file(os.path.join(data_path, "ssd_dupl_to_delete.shp"))
print(df1.head())

    index_x      name    lat_x     long_x    distance  \
13        3  LOMIYEGA  4.57257  31.958799  315.162902   
17     5122  LOMIYEGA  4.57000  31.959999  315.162902   
76    19405    MURGAN  8.06795  27.857599  372.469294   
82    29668    MURGAN  8.06997  27.854900  372.469294   
85    29668    MURGAN  8.06997  27.854900  414.333243   

                             MID  index_col_x                  geometry  
13  31.959399219954.571285009845            3  POINT (31.95880 4.57257)  
17  31.959399219954.571285009845         5122  POINT (31.96000 4.57000)  
76     27.856249818.068960189715        19405  POINT (27.85760 8.06795)  
82     27.856249818.068960189715        29668  POINT (27.85490 8.06997)  
85   27.85305022998.069630145965        29668  POINT (27.85490 8.06997)  


# Delete duplicated name locations within threshold distance and save cleaned layer

In [9]:
df2 = df1.sort_values(['name', 'MID'], ascending = True)
df3 = df2.drop_duplicates(subset='MID', keep="first")

df4 = df3.sort_values(['index_col_x'], ascending = True)
df5 = df4.drop_duplicates(subset='index_col_x', keep="first")
df5.shape


(716, 8)

In [10]:
dfm = pd.merge(left=df, right=df5, how = 'left', left_on ='index_col',right_on='index_col_x')
dfm.shape

(30677, 31)

In [11]:
dfm.rename(columns={'name_x':'name'}, inplace = True)
df_select = dfm.loc[dfm['MID'].isnull()]
df_select.shape

(29961, 31)

In [12]:
df3 = df_select.iloc[:, :19]
gdf = gpd.GeoDataFrame(df3, geometry=gpd.points_from_xy(df3.long, df3.lat))
gdf.to_file(os.path.join(data_path, "ssd_dupl_500m_deleted.shp"))